## Demo code for Bayesian analysis

We first simulate the posterior distribution for the "heads probability" of a coin. The prior and posterior distributions are Beta distributions. 

Then, we simulate the posterior distribution for the weightings of a "three sided die" -- a multinomial on three outcomes. Here the prior and posterior distributions are Dirichlet.


In [2]:
import os, gzip
import numpy as np
import matplotlib.pyplot as plt


## Bayesian calculation for flipping a coin

Here we illustrate the basic Bayes inference scheme when the probability of heads is uncertain.


In [7]:
%matplotlib qt
from scipy.special import gamma
from scipy import random
from scipy.stats import beta

# we dont know theta, it could be anywhere between 0 and 1, but
# we suspect it might be 0.5
theta = np.linspace(0,1,num=500) 
fig = plt.figure(1)
plt.ion()

a0 = 1*2
b0 = 1*3
sample_size = 100
p = 0.8 # in this case the true parameter is 0.8 (which we don't know)
prior = beta.pdf(theta, a0, b0)
# we assume a prior beta distrbution over theta with two parameters

# for about 21 tries  
# 0, 50, 100, 150, 200, 250, 300 ... 1000
for n in np.arange(0, 1001, 50):
    
    # sample from a coin with probability of heads p
    x = random.multinomial(n, pvals=[p, 1-p]) 
    # a multinomial is a generalization of a bionomial
    # n is how many experiments to do (e.g. toss the coin 50 times)
    
    # update alpha and beta parameters
    a = a0 + x[0]
    b = b0 + x[1]
    posterior = beta.pdf(theta, a, b)
    
    # sample from the posterior
    posterior_sample = np.random.beta(a, b, size=sample_size)
    
    # plot everything
    plt.cla()
    plt.title('n=%d' % n)
    plt.plot(theta, prior, linewidth=3, color='red', label='prior')
    plt.plot(theta, posterior, linewidth=1, color='black', alpha=.8, label='posterior')
    plt.xlim((0, 1))
    plt.hist(posterior_sample, density=True, color='lightblue', alpha=.8)
    plt.scatter([p], [0], color='black', label='true parameter')
    plt.legend(loc='upper left')
    plt.pause(0.5)
    
#     break

plt.close(fig)
plt.ioff()


## Dirichlet distributions

And here we do the same but for a "three-sided die", where the weighting of the die is uncertain. 


In [8]:
import warnings
warnings.filterwarnings("ignore")
from dirichlet import *


fig = plt.figure(1)
plt.ion()

alpha0 = [2, 4, 5]
prior = Dirichlet(alpha0) 
# we have a prior distrbution which is Dirchilet
model = [.333, .333, .333]

# for 10, 20, 30, 40, 50, 60, 70, 80, 90, 100
for n in np.arange(10, 101, 10):
    
    x = random.multinomial(n, pvals=model)
    alpha = alpha0 + x
    
    print(x)
    print(alpha0)
    print(alpha)
    
    break

    
    posterior = Dirichlet(alpha)
    
    # plot prior
    plt.title('n=%d' % n)
    plt.subplot(131)
    plt.cla()
    plt.title('prior' % n)
    draw_pdf_contours(prior)
    
    # plot posterior
    plt.subplot(132)
    plt.cla()
    plt.title('posterior n=%d' % n)
    draw_pdf_contours(posterior)

    # plot sample from posterior
    plt.subplot(133)
    plt.cla()
    plt.title('sample from posterior')
    plot_points(posterior.sample(sample_size), markersize=5)
    plt.pause(0.5)
    
plt.close(fig)
plt.ioff()

[3 4 3]
[2, 4, 5]
[5 8 8]


## Sampling from a mixture

In the following bit of code, we illustrate how to sample from a mixture model. In this case we're sampling from a mixture of two Gaussians.


In [ ]:
%matplotlib inline
import scipy.stats as sps

sample = []
weight = [0.3, 0.7]

for i in np.arange(1000):
    x = random.multinomial(1, pvals=weight)
    y0 = np.random.normal(loc=-2)
    y1 = np.random.normal(loc=2)
    y = x[0]*y0 + x[1]*y1
    sample.append(y)

x = np.linspace(-6, 6, num=200)
mix = weight[0]*sps.norm(loc=-2).pdf(x) + weight[1]*sps.norm(loc=2).pdf(x)
_ = plt.hist(sample, density=True, color='lightblue', alpha=.8)
_ = plt.plot(x, mix, color='black')